# Bayesian Theory Of Mind Problem Set


1. [Building The Model](#Distance-Calc)
  1. [Initializing The Model](#init)
  1. [Distance-Calc](#Distance-Calc)
  2. [Getting Points Within The Range](#Within-Range)
2. [The Mind Model](#Mind-Model)
  1. [Updating Beliefs](#updating-beliefs)
  2. [Defining Transition and Action](#transition-matrix)
  3. [Best Policy](#best-policy)
  4. [Inferring Intent](#inferring-intent)
3. [Analysis of Bayesian Theory Of Mind](#analysis)

## Building The Model

<img src="scenario.png"/>


Before we start constructing a way to infer intent, we first need to find a way to represent our "world", the space in which we will infer intent. In this case, our world is a 15x15 square grid that our agent (in this case, Mark Watney) can traverse around. Scattered in known locations on the grid are resources A B and C. Mark is trying to retrieve these resources, but we're not sure which one he wants. This is where the intent comes in!

To be more technical and specific, we will write some class Mind that keeps track of our model's estimation of Mark's intent, belief about the world, and how likely he's going for a certain resource given his location.

Below, we'll initialize this Mind class, then dive a bit deeper into how to use it.



### Initialize The Mind (5 pts) <a id="temporal-word-problem"/>

Our first step is to initialize the Bayesian Theory of Mind. Below is an implementation of the class we'll use to model our BToM, more in particular the init.



In [131]:
class Mind:
    def __init__(self):
        # where resources are
        self.world_loc = [(10,0), (0,9), (6,10)]
        self.map_length = 15
        self.world_state = 'ABC'

        # possible assignments to world
        self.beliefs_worlds = ['ABC', 'ACB', 'BAC', 'BCA', 'CAB', 'CBA']
        # belief that the orientation is the above
        self.beliefs = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]

        self.intents = {'A': 1/3,
                        'B': 1/3,
                        'C': 1/3}

        self.state = (0,0)

        self.states = []
        for i in range(self.map_length):
            for j in range(self.map_length):
                self.states.append((i,j))

        self.gamma = 0.5
        

Let's talk about the initialization below. Most of the initialization is self explanatory. The world and map length gives us what the map looks like, and the "actual world" gives us where the resources really are. In this case, it's "ABC", which means that A is at (10,0), B is at (0,9), and C is at (6,10).

belief_worlds gives us all possible assignments to worlds. state tells us where we are in the world, whereas states tell us all possible states (we use this and gamma in value iteration, it's not quite the focus of this pset so don't worry too much about it)

Then, we initialize the self.beliefs and self.intents. Notice that all the probabilites are initialized to the same thing. What is this distribution called? What is the benefit of initializing our beliefs and intents in this way, and can you think of any other ways to do it?  

ANSWER: The distribution is a uniform distribution. Any attempt at improving the model could be considered an answer for the second part, for example, taking existing intents and factoring that into our initial distribution, or if the observer is aware that the agent has some beliefs, that could also be taken into account.

### Distance Calculation (5 pts) <a id="distance-calc"/>

In order for our model to understand anything, we need to be able to calculate distances between objects on our field. Write a function here to calculate the distance between two points p1 and p2 (both tuples of the form x,y)

In [132]:
# helper to get distance
def get_distance(p1, p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**.5

Let's test that our Distance Calculation Works as expected

In [133]:
from nose.tools import assert_equal

assert_equal(get_distance((1,0), (2,0)), 1)
assert_equal(get_distance((1,1), (2,2)), (2)**0.5)
assert_equal(get_distance((1,0), (1,2)), 2)

print ("success")


success


### Getting Points Within A Range (10 pts) <a id="within-range"/>

Now, our next step is to be able to find the points that Mark can see when he is at some location. Implement the method below, that given a Mind object and location, returns all the resources that Mark can see. (5 squares away in x or y direction)

In [134]:
def within_range(mind, position):
    """ 
    Given a mind object and location, outputs the (x,y) coordinates of resources near the location
    
    Input: mind - a mind object which contains location of all the resources on map
           position - current position on map
    Output: locs - a list of locations (in x,y) available from current position
    """
    # should return resource positions that are visible from current position
#     locs = []
    
#     return locs

    locs = []
    for i in range(len(mind.world_loc)):
        loc = mind.world_loc[i]
        if abs(position[0]-loc[0])<=5 and abs(position[1]-loc[1])<=5:
            locs.append(loc)
    return locs


In [135]:
from nose.tools import assert_equal

my_mind = Mind()

assert_equal(within_range(my_mind, (8,0)), [(10, 0)])
assert_equal(within_range(my_mind, (0,0)), [])
assert_equal((6,10) in within_range(my_mind, (3,9)) and (0,9) in within_range(my_mind, (3,9)), True)
print ("success")


success


## The Mind Model

Next, we need to actually write some useful functions that will use the mind model to get intent


### Updating Beliefs (20 pts) <a id="updating-beliefs"/>

In this section, we'll work on updating what our BToM thinks that Mark believes.

The function below is passed a mind instance and the current state. We should update our probability distribution of what we think Mark believes given this new information.

Here are the steps you should follow:
- Obtain all locations of resources that are within range of our state (use a function you made!)
For each of the locations you will then:
- Get the resource that is at that location in the actual world
- Go through all the 6 possible worlds, keep track of which are consistent and inconsistent with the resource being in that location.
- For each of the consistent worlds, multiply the previous belief in that world by 0.9 over the number of consistent worlds.
- Similarly for the inconsistent worlds, multiply the previous belief in that world by 0.1 over the number of inconsistent worlds.
- Normalize the new beliefs

Hint: If the position of the resource we just discovered matches some possible state of the world, Mark is also much more likely to think that that is the true state of the world.



In [136]:
def beliefs_update(mind, state):
    "Update beliefs"
    near_locations = within_range(mind, state)

    # figure out which locations you can see
    # update beliefs for the ones close to you

    #redistribute beliefs

    for loc in near_locations:
        i = mind.world_loc.index(loc)
        resource = mind.world_state[i]
#         print ("observed resource %s" % resource)
        consistently_observed_worlds = []
        inconsistently_observed_worlds = []
        for j in range(len(mind.beliefs_worlds)):
            world = mind.beliefs_worlds[j]
            if world[i] == resource:
                consistently_observed_worlds.append(j)
            else:
                inconsistently_observed_worlds.append(j)
        factor = len(consistently_observed_worlds)
        sum_of_consistently_observed = sum([mind.beliefs[i] for i in consistently_observed_worlds])
        sum_of_inconsistently_observed = sum([mind.beliefs[i] for i in inconsistently_observed_worlds])
        for i in range(len(mind.beliefs)):
            if i in consistently_observed_worlds:
                mind.beliefs[i] = 0.9 * mind.beliefs[i]/sum_of_consistently_observed
            else:
                mind.beliefs[i] = 0.1 * mind.beliefs[i]/sum_of_inconsistently_observed

        mind.beliefs = [float(i)/sum(mind.beliefs) for i in mind.beliefs]
#         print("new beliefs %s " % mind.beliefs)


In [137]:
from nose.tools import assert_equal, assert_true

my_mind = Mind()

beliefs_update(my_mind, (10,0))
assert_equal(my_mind.beliefs[0], 0.45)
beliefs_update(my_mind, (0,9))
assert_true(abs(my_mind.beliefs[0]-0.85)<0.01)

print ("success")


success


### Defining Transition and Actions (20 pts) <a id="transition-matrix"/>

In addition to wanting to figure out Mark's intent, our model also models Mark's mind in the context of a planning problem as well. We'd like to know what action Mark wants to take next. Although this also includes the value iteration algorithm as well as getting the best policy (the best actions he should take), we'll be defining just some of the functions necessary to get the best policy.

We'll be implementing a function, transition, that giving a state and an action, returns a list of (result state, probability) pairs. 

Hint: We included two helper function, get_next_state, that gives the next state that would be reached by taking some action, and actions, which gets the next set of possible functions. Use them to help implement transition.

In [138]:
def get_next_state(mind, state, action):
    """
    Helper function. Given state and action, gives next state that would
    be reached. Returns None if off map.
    """
    next_state = None
    x = state[0]
    y = state[1]

    if action == 'right':
        if 0<=(x+1)<mind.map_length and 0<=y<mind.map_length:
            next_state = (x+1,y)
    elif action == 'left':
        if 0<=(x-1)<mind.map_length and 0<=y<mind.map_length:
            next_state = (x-1,y)
    elif action == 'up':
        if 0<=x<mind.map_length and 0<=(y-1)<mind.map_length:
            next_state = (x,y-1)
    elif action == 'down':
        if 0<=x<mind.map_length and 0<=(y+1)<mind.map_length:
            next_state = (x,y+1)

    return next_state


def actions(mind, state):
    "Set of actions that can be performed in this state."
    actions = []
    x = state[0]
    y = state[1]

    if 0<=(x+1)<mind.map_length and 0<=y<mind.map_length:
        actions.append('right')
    if 0<=(x-1)<mind.map_length and 0<=y<mind.map_length:
        actions.append('left')
    if 0<=x<mind.map_length and 0<=(y-1)<mind.map_length:
        actions.append('up')
    if 0<=x<mind.map_length and 0<=(y+1)<mind.map_length:
        actions.append('down')

    return actions

    
def transition(mind, state, action):
    """
    Transition model.  From a state and an action, return a list
    of (result-state, probability) pairs.
    """
    pos_actions = actions(mind, state)
    pairs = []

    action_valid = False
    next_state = get_next_state(mind, state, action)

    x = state[0]
    y = state[1]

    if next_state:
        pairs.append((next_state, 0.9))
        remaining = len(pos_actions) - 1
        prob = 0.1 / remaining
        for a in pos_actions:
            if a != action:
                pairs.append((get_next_state(mind, state, a), prob))
    else:
        prob = 1.0 / len(pos_actions)
        for a in pos_actions:
            pairs.append((get_next_state(mind, state, a), prob))

    return pairs


In [139]:
from nose.tools import assert_equal

my_mind = Mind()
print(transition(my_mind, (1,1), 'down'))
assert_equal(transition(my_mind, (1,1), 'down'), [((1, 2), 0.9), ((2, 1), 0.03333333333333333), ((0, 1), 0.03333333333333333), ((1, 0), 0.03333333333333333)]
)

print(transition(my_mind, (4,3), 'right'))
assert_equal(transition(my_mind, (4,3), 'right'), [((5, 3), 0.9), ((3, 3), 0.03333333333333333), ((4, 2), 0.03333333333333333), ((4, 4), 0.03333333333333333)]
)


print ("success")


[((1, 2), 0.9), ((2, 1), 0.03333333333333333), ((0, 1), 0.03333333333333333), ((1, 0), 0.03333333333333333)]
[((5, 3), 0.9), ((3, 3), 0.03333333333333333), ((4, 2), 0.03333333333333333), ((4, 4), 0.03333333333333333)]
success


### Best Policy <a id="best-policy"/>

Again, Mark's problem, to him, can be seen as a planning problem, since each step, he wants to choose the best action that will take him to the goal. We can calculate the best policy below, using reward and value iteration as mentioned in lecture. Just run our cases and examine the output!

In [140]:
def reward(mind, state, world):
    "Return a numeric reward for this state."
    if (state in mind.world_loc):
        i = mind.world_loc.index(state)
        resource = world[i]
        return 10*mind.intents[resource]
    else:
        return -1
    
def value_iteration(mind, epsilon=0.001):
    "Solving by value iteration."

    U1 = dict([(s, 0) for s in mind.states])
    R, T, gamma = reward, transition, mind.gamma

    for i in range(10):
        U = U1.copy()
        delta = 0
        for w_i in range(len(mind.beliefs_worlds)):
            for s in mind.states:
                U1[s] = R(mind, s, mind.beliefs_worlds[w_i]) + gamma * max([sum([mind.beliefs[w_i] * p * U[s1] for (s1, p) in T(mind, s, a)])
                                            for a in actions(mind, s)])
                delta = max(delta, abs(U1[s] - U[s]))
        if delta < epsilon * (1 - gamma) / gamma:
             return U
    return U

def argmax(keys, f):
    "Helper function to get argmax."
    return max(keys, key=f)

def expected_utility(mind, action, state, U):
    "The expected utility of doing a in state s, according to the MDP and U."
    return sum([p * U[s1] for (s1, p) in transition(mind, state, action)])

def best_policy(mind, U):
    """
    Given an MDP and a utility function U, determine the best policy,
    as a mapping from state to action.
    """
    pi = {}
    for s in mind.states:
        pi[s] = argmax(actions(mind,s), lambda a:expected_utility(mind, a, s, U))
    return pi


In [141]:
my_mind = Mind()

beliefs_update(my_mind, (1,0))
U = value_iteration(my_mind)
policy = best_policy(my_mind, U)
print (policy[(0,1)])
print (policy[(2,2)])
print (policy[(9,3)])

right
right
up


### Inferring Intent (20 pts) <a id="inferring-intent"/>

Last thing! We need to put it together and infer the intent of Mark using the functions we've defined!

In this function, we're passed in a Mind object. Using it, we should get the total probability (normalized) that Mark's intent is to get a particular resource.

Steps:
- Based on the action Mark takes next, get the next state
- For each of the resource positions in the actual world orientation, get the difference in distance Mark is from each of the resource positions based on the state before and after his action. Note: some of these may be negative.
- For each of the resources, taking into account the belief distribution in all the different worlds, we want to calculate a score, based on the distance it is from Mark in that world and the belief in that world (to simplify, you can simply multiply the distance by the belief).
- Update the old intents by adding this score, discounted by a factor of 0.3. Ensure that the values do not exceed 1 or go below 0.
- Finally, normalize the new intents.

More specifically, you should mutate mind.intents and update it with the new intents! We've already implemented the step for receiving an observation, and updating other propertiess, but now you must finish it with update_intents.


In [142]:
def intents_update(mind, action):
    "Update intents."
    """
        belief: [Pr(ABC), Pr(ACB), Pr(BAC), Pr(BCA), Pr(CAB), Pr(CBA)]
        state: current location on the grid
        action: action he takes
    """

    next_state = get_next_state(mind, mind.state, action)

    # difference in distance that this action takes us, or how much closer or farther
    # we get from the 3 resource locations
    dists = []

    for resource_pos in mind.world_loc:
        dist1 = get_distance(resource_pos, mind.state)
        dist2 = get_distance(resource_pos, next_state)
        diff_dist = dist1-dist2
        dists.append(diff_dist)

    new_intents_scores = {'A': 0, 'B': 0, 'C': 0}

    for i in range(len(mind.beliefs_worlds)):
        world = mind.beliefs_worlds[i]
        belief = mind.beliefs[i]
        for i in range(3):
            r = world[i]
            new_intents_scores[r] += dists[i] * belief

    new_intents = {}
    factor = 0.3
    sum_new_intents = 0
    for r in mind.intents:
        new_intents[r] = max(mind.intents[r] + factor * new_intents_scores[r], 0)
        sum_new_intents += new_intents[r]

    mind.intents = {'A': new_intents['A']/sum_new_intents, 'B': new_intents['B']/sum_new_intents, 'C': new_intents['C']/sum_new_intents}


In [143]:
def receive_observation(mind, action):
    "Receive observation, update model, and get updated intents."
    mind.state = get_next_state(mind, mind.state, action)
    beliefs_update(mind, mind.state)
#     U = value_iteration(mind)
#     policy = best_policy(mind, U)
    intents_update(mind, action)
    print ('intents: ', mind.intents)

In [144]:
# testing
# goes to A but turns back around.
my_mind = Mind()
print("Mark goes to A, but turns back around.")
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'left')
receive_observation(my_mind, 'left')
receive_observation(my_mind, 'left')
receive_observation(my_mind, 'left')

print ("---------------------------")

# goes to A, but then starts going to C.
my_mind = Mind()
print("Mark goes to A, but then starts going to C.")
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'right')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')
receive_observation(my_mind, 'down')

Mark goes to A, but turns back around.
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents:  {'A': 0.5146189448302096, 'B': 0.24269052758489512, 'C': 0.24269052758489512}
intents:  {'A': 0.33483652227777927, 'B': 0.33258173886111037, 'C': 0.33258173886111037}
intents:  {'A': 0.09380125471630178, 'B': 0.45309937264184913, 'C': 0.45309937264184913}
intents:  {'A': 0.0, 'B': 0.5, 'C': 0.5}
intents:  {'A': 0.0, 'B': 0.5, 'C': 0.49999999999999994}
---------------------------
Mark goes to A, but then starts going to C.
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents:  {'A': 0.3333333333333333, 'B': 0.3333333333333333, 'C': 0.3333333333333333}
intents: 

Qualitatively describe what you saw when we ran our simulation. How did the intents change as we progressed through time? Can you explain what happened, and was it consistent with the bayesian theory of mind? Try to use how we used belief in your answer. (10 points)

In the first simulation, we saw Mark approach a resource, and then leave. What does this say about his intents and beliefs at that point? (10 points)

ANSWER: TODO

## Analysis of Bayesian Theory of Mind (10 pts) <a id="distance-calc"/>

In the box below, highlight some of the advantages and disadvantages of Bayesian Theory of Mind. This is open ended! What makes it different from other approaches, and why does that make it better? Are there any drawbacks you can think of?

ANSWER